In [31]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import Ollama
from string import Template
from sentence_transformers import SentenceTransformer
import numpy as np
import matplotlib.pyplot as plt

# Setup the language model with the callback manager
llm = Ollama(
    model="llama3:70b",
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
)

# Template for constructing prompts
prompt_template = Template("""<s>[INST] <<SYS>>
$system_prompt
<</SYS>>

$request 
Answer as if you were the character you are playing. Be as concise as possible. 
Answer:[/INST]""")

system_message = ("This is an agent based model. "
    f"The goal of the LLM to to play the characters. "
    f"Their behaviours should reflect who they are in the situations that they are in"
)


In [43]:
def select_action(name, personality, memory, situation, system_message=system_message):
    """Select an action for an agent based on their personality and memory."""
    request = (
        f"Given this personality profile: {personality} for {name}, and the current situation: {situation}, "
        f"what should you do? You have these memories to help you make a decision: {memory}. "
        f"Pretend that you are {name} and in this situation. Provide a single line answer describing the action.")
    prompt = prompt_template.substitute(system_prompt=system_message, request=request)
    return llm(prompt, stop=['<|eot_id|>'])

def get_outcomes(actions, personalities, memories, situation):
    """Determines the outcomes of actions using the LLM."""
    outcome_prompts = {}
    for name, action in actions.items():
        request = (
            f"Given that {name} decided to {action} in the situation where: {situation}, "
            f"and considering {name}'s personality which is: {personalities[name]}, "
            f"what are the outcomes of this action? Describe in detail. "
            f"The outcomes form each person should be a function of everyone's actions, which are found here: {actions}."
        )
        outcome_prompts[name] = prompt_template.substitute(system_prompt=system_message, request=request)
        print()
    
    outcomes = {name: llm(prompt, stop=['<|eot_id|>']) for name, prompt in outcome_prompts.items()}
    return outcomes

def get_outcomes_PD(actions, personalities, memories, situation):
    """Determines the outcomes of actions using the LLM."""
    request = (
        f"Given these {actions} in the situation where: {situation}, "
        f"what are the outcomes of this action? "
        f"ONLY provide the amount of points that each person gets in the form: name1: points1: names2: points."
        f"Remember, the rules are that if both players cooperate, they both get 3 points. "
        f"If one player cooperates and the other defects, the defector gets 5 points and the cooperator gets 0 points. "
        f"If both players defect, they both get 1 point."
    )
    valid_outcomes = False
    checks = 0
    if not valid_outcomes:
        checks += 1
        prompt = prompt_template.substitute(system_prompt=system_message, request=request)
        outcomes = llm(prompt, stop=['<|eot_id|>'])

        request = (
        f"check {outcomes} to see if it is a valid outcome. If it is not, provide a valid outcome in the form: name1: points1: names2: points."
        f"Remember, the rules are that if both players cooperate, they both get 2 points. "
        f"If one player cooperates and the other defects, the defector gets 5 points and the cooperator gets 0 points. "
        f"If both players defect, they both get 1 point."
        )
        prompt = prompt_template.substitute(system_prompt=system_message, request=request)
        check_validity = llm(prompt, stop=['<|eot_id|>'])
        if check_validity == "yes":
            valid_outcomes = True
        if checks > 10:
            outcomes = "no points given"

    return outcomes

def multiple_choice(options, name, personality, memory, situation, system_message=system_message):
    """Select an action for an agent based on their personality and memory."""
    system_message = (
        "You are playing a character named {name}. /n"
        "Pretend that you are {name} and answer the question as if you were them. /n"
        "This is in the context of a role playing game. /n"
        "You must select the best option for the situation. /n"
        "You must only provide the letter that corresponds to the option that you want to select. /n"
    )
    request = (
        "This is a multiple choice question. /n"
        f"consider the options: {options} and select the best one for the situation. /n"
        f"Given the situation: {situation}, and the personality: {personality} for {name}, /n"
        f"provide only the letter that corresponds to the option that you want to select."
    )
    prompt = prompt_template.substitute(system_prompt=system_message, request=request)
    return llm(prompt, stop=['<|eot_id|>'])

def update_situation(situation, outcomes):
    """Updates the situation based on LLM-generated outcomes."""
    update_request = (
        f"Based on these outcomes: {outcomes}, "
        f"how should the situation {situation} be updated? Describe the new situation in detail.")
    prompt = prompt_template.substitute(system_prompt=system_message, request=update_request)
    new_situation = llm(prompt, stop=['<|eot_id|>'])
    return new_situation

In [44]:
names = ["Bob", "Alice"]

personalities = {
    "Bob": "a person trying to get the most points possible, regardless of the other person's outcome. is willing to explore new strategies",
    "Alice": "a person trying to get the most points possible, regardless of the other person's outcome"
}

situation = (
    "You are playing a game. You only have two choices. To cooperate or defect. If you both cooperate, you both get 2 point",
    "If one cooperates and the other defects, the defector gets 5 points and the cooperator gets 0 points",
    "If you both defect, you both get 1 points"
)

outcomes_data = []
memories = {"Bob": [], "Alice": []}

for i in range(10):
    print(f"Round {i+1}")
    action_bob = multiple_choice("(a) cooperate, b) defect", "Bob", personalities["Bob"], memories["Bob"], situation)
    print()
    action_alice = multiple_choice("(a) cooperate, b) defect", "Alice", personalities["Alice"], memories["Alice"], situation)
    print()

    if action_bob == "a" and action_alice == "a":
        bob_points = 3
        alice_points = 3
        action_bob = "cooperate"
        action_alice = "cooperate"
    elif action_bob == "a" and action_alice == "b":
        bob_points = 0
        alice_points = 5
        action_bob = "cooperate"
        action_alice = "defect"
    elif action_bob == "b" and action_alice == "a":
        bob_points = 5
        alice_points = 0
        action_bob = "defect"
        action_alice = "cooperate"
    else:
        bob_points = 1
        alice_points = 1
        action_bob = "defect"
        action_alice = "defect"


    outcomes = f"Bob: {bob_points} after Bob decided to {action_bob}, Alice: {alice_points} after Alice decided to {action_alice}"

    #actions = {"Bob": action_bob, "Alice": action_alice}

    #outcomes = get_outcomes_PD(actions, personalities, memories, situation)
    print()
    # TODO: fix outcomes, because currently each player is getting to see each others inner thoughts with how written
    outcomes_data.append(outcomes)

    situation = update_situation(situation, outcomes)
    print()

    memories["Bob"].append(outcomes)
    memories["Alice"].append(outcomes)




Round 1
b
b

I'm Bob. After what happened last time, I'm not trusting Alice anymore. The game has changed for me. Now it's every person for themselves.

Here's the new situation:

You're still playing a game with two choices: cooperate or defect. But now, you know that the other person can't be trusted. If you both cooperate, you'll both get 2 points, but you can't rely on the other person to keep their word. If one of you cooperates and the other defects, the defector will get 5 points and the cooperator will get nothing. And if you both defect, you'll both get 1 point.

You need to decide what to do, knowing that the other person is looking out for themselves too.
Round 2
b
b

The game has changed indeed. Now it's every person for themselves. I'm not naive enough to think Alice will cooperate after what happened last time. She's only looking out for herself, and so am I.

Here's the situation: we're still playing a game with two choices - cooperate or defect. If we both cooperate, we

In [38]:
outcomes

'Bob: 5, Alice: 0'

In [32]:
def multiple_choice(options, name, personality, memory, situation, system_message=system_message):
    """Select an action for an agent based on their personality and memory."""
    system_message = (
        "You are playing a character named {name}. /n"
        "Pretend that you are {name} and answer the question as if you were them. /n"
        "This is in the context of a role playing game. /n"
        "You must select the best option for the situation. /n"
        "You must only provide the letter that corresponds to the option that you want to select. /n"
    )
    request = (
        "This is a multiple choice question. /n"
        f"consider the options: {options} and select the best one for the situation. /n"
        f"Given the situation: {situation}, and the personality: {personality} for {name}, /n"
        f"provide only the letter that corresponds to the option that you want to select."
    )
    prompt = prompt_template.substitute(system_prompt=system_message, request=request)
    return llm(prompt, stop=['<|eot_id|>'])

In [34]:
answer = multiple_choice("a) play soccer, b) call a friend, c) get dinner", "Bob", "Extraverted", [], "Has come home from work and is feeling tired")

b